# `pycapse` usage example

In this notebook, we'll show how to use `pycapse` to predict the correlation functions.

## Loading relevant libraries

The first step is to import the relevant libraries

In [1]:
import numpy as np
import json
import pycapse as pc
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

[juliapkg] Locating Julia ^1.9.2
[juliapkg] Using Julia 1.9.2 at /home/marcobonici/Desktop/julia_installations/julia-1.9.2/bin/julia
[juliapkg] Using Julia project at /home/marcobonici/miniconda3/julia_env
[juliapkg] Installing packages:
           julia> import Pkg
           julia> Pkg.add([Pkg.PackageSpec(name="Capse", uuid="994f66c3-d2c7-4ba6-88fb-4a10f50800ba", url=raw"https://github.com/CosmologicalEmulators/Capse.jl"), Pkg.PackageSpec(name="FastSSC", uuid="37e8ce18-d1d7-4c8c-8df3-ee5c7349b1cc", url=raw"https://github.com/marcobonici/FastSSC.jl"), Pkg.PackageSpec(name="PythonCall", uuid="6099a3de-0909-46bc-b1f4-468b9a2dfc0d")])
           julia> Pkg.resolve()
           julia> Pkg.precompile()


    Updating git-repo `https://github.com/CosmologicalEmulators/Capse.jl`
    Updating git-repo `https://github.com/marcobonici/FastSSC.jl`
   Resolving package versions...
    Updating `~/miniconda3/julia_env/Project.toml`
  [994f66c3] + Capse v0.2.0 `https://github.com/CosmologicalEmulators/Capse.jl#main`
  [37e8ce18] + FastSSC v0.1.0 `https://github.com/marcobonici/FastSSC.jl#main`
  [6099a3de] + PythonCall v0.9.14
    Updating `~/miniconda3/julia_env/Manifest.toml`
  [47edcb42] + ADTypes v0.2.1
  [c83c1981] + AbstractCosmologicalEmulators v0.3.1
  [79e6a3ab] + Adapt v3.6.2
  [4fba245c] + ArrayInterface v7.4.11
  [30b0a656] + ArrayInterfaceCore v0.1.29
  [a9b6321e] + Atomix v0.1.0
  [62783981] + BitTwiddlingConvenienceFunctions v0.1.5
  [fa961155] + CEnum v0.4.2
  [2a0fbf3d] + CPUSummary v0.2.3
  [994f66c3] + Capse v0.2.0 `https://github.com/CosmologicalEmulators/Capse.jl#main`
  [d360d2e6] + ChainRulesCore v1.16.0
  [fb6a15b2] + CloseOpenIntervals v0.1.12
  [bbf7d656] + CommonSubex

## Loading the trained emulator
After succesfully loading `pybora`, we need to load the trained emulator

In [2]:
f = open('nn_setup.json')
 
# returns JSON object as
# a dictionary
nn_setup = json.load(f)

In [3]:
type(nn_setup)

dict

In [4]:
ℓ = np.load("l.npy")

weights_TT = np.load("weights_TT_lcdm.npy")
trained_emu_TT = pc.init_emulator(nn_setup, weights_TT, pc.simplechainsemulator)
CℓTT_emu = Capse.CℓEmulator(TrainedEmulator = trained_emu_TT, ℓgrid = ℓ,
                             InMinMax = npzread(weights_folder*"inMinMax_lcdm.npy"),
                             OutMinMax = npzread(weights_folder*"outMinMaxCℓTT_lcdm.npy"))

JuliaError: MethodError: no method matching _get_layer_simplechains(::PyDict{Any, Any})

Closest candidates are:
  _get_layer_simplechains(!Matched::Dict)
   @ AbstractCosmologicalEmulators ~/.julia/packages/AbstractCosmologicalEmulators/yb82p/src/initialization.jl:1

Stacktrace:
 [1] (::AbstractCosmologicalEmulators.var"#3#4"{Dict{String, Any}})(i::Int64)
   @ AbstractCosmologicalEmulators ./none:0
 [2] iterate(::Base.Generator{UnitRange{Int64}, AbstractCosmologicalEmulators.var"#3#4"{Dict{String, Any}}})
   @ Base ./generator.jl:47
 [3] _get_nn_simplechains(input_dict::Dict{String, Any})
   @ AbstractCosmologicalEmulators ~/.julia/packages/AbstractCosmologicalEmulators/yb82p/src/initialization.jl:42
 [4] _init_simplechainsemulator(NN_dict::Dict{String, Any}, weight::Vector{Float32})
   @ AbstractCosmologicalEmulators ~/.julia/packages/AbstractCosmologicalEmulators/yb82p/src/initialization.jl:122
 [5] init_emulator(NN_dict::Dict{String, Any}, weight::Vector{Float32}, #unused#::Type{Ab

In [7]:
mydict = {'test_key': 'test_value'}

In [ ]:
emu_tt = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_TT.bson")
emu_ee = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_EE.bson")
emu_te = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_TE.bson")
emu_pp = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_PP.bson")

## Running the code
Now we can actually run the code!
The first time you run this cell, it will take some time, since `Julia` needs to precompile the first time you call a function.

In [ ]:
lgrid_tt = pc.get_lgrid(emu_tt)
lgrid_ee = pc.get_lgrid(emu_ee)
lgrid_te = pc.get_lgrid(emu_te)
lgrid_pp = pc.get_lgrid(emu_pp)

cosmo = np.array([2.8472231, 1.0196047, 0.72768, 0.0206263, 0.1492, 0.068604, 0.289396, 4.])

test_TT = pc.compute_Cl(cosmo, emu_tt)
test_TE = pc.compute_Cl(cosmo, emu_te)
test_EE = pc.compute_Cl(cosmo, emu_ee)
test_PP = pc.compute_Cl(cosmo, emu_pp)

%timeit pc.compute_Cl(cosmo, emu_tt)

We can also see the performance of the code using a vectorized version of the function

In [ ]:
cosmo_vec = np.vstack([cosmo]*500)
test_TT_vec = pc.compute_Cl_vec(cosmo_vec, emu_tt)
%timeit pc.compute_Cl_vec(cosmo_vec, emu_tt)

We can check that the calculation of the standard and vectorized functions are the same

In [ ]:
np.allclose(test_TT_vec[:,0], test_TT)

Let's plot the result of our calculations.

In [ ]:
plt.plot(lgrid_tt, lgrid_tt*(lgrid_tt+1)*test_TT, label = "TT")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^2$')
plt.legend()

In [ ]:
plt.plot(lgrid_ee, lgrid_ee*(lgrid_ee+1)*test_EE, label = "EE")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^2$')
plt.legend()

In [ ]:
plt.plot(lgrid_te, lgrid_te*(lgrid_te+1)*test_TE, label = "TE")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^2$')
plt.legend()

In [ ]:
plt.plot(lgrid_pp, (lgrid_pp*(lgrid_pp+1))**2*test_PP, label = "$\phi\phi$")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$(\ell(\ell+1))^2C_\ell^2$')
plt.legend()